Execute the following cell to render the README.md inside the notebook:

In [3]:
from IPython.display import display, Markdown
with open('README.md', 'r') as fh:
    content = fh.read()
display(Markdown(content))

# Binfit Tutorial

Tutorial for the [binfit](https://stash.desy.de/users/sutclw/repos/binfit)
package developed for template fits in Belle II analyses.

## Background information on`binfit` and relation to `TemplateFitter`

`binfit` is a python package for performing template fits in pure python
developed and maintained by [William
Sutcliffe](mailto:william.sutcliffe08@gmail.com).

Its code is based in large parts on [Maximillian Welsch](mailto:mwelsch@uni-bonn.de)'s `TemplateFitter` package,
which is also openly [available on github](https://github.com/welschma/TemplateFitter).

Another fork of the `TemplateFitter` package is being actively developed by [Felix Metzner](mailto:felix.metzner@kit.edu), also [on github](https://github.com/FelixMetzner/TemplateFitter). As far as I understand he extends it generalizes the template fitter, e.g. with support for arbitrary dimensions, adaptive binning.

## Other references / tutorials

An already existing example notebook can be found in the [binfit/examples/](https://stash.desy.de/users/sutclw/repos/binfit/browse/binfit/examples) directory of the packages. It requires you to clone binfit (see *Installation* section below). I will take inspiration from that.

There is a `docs/` directory which is meant to contain sphinx package documentation (what basf2 uses) in RestructuredText (rst format), but the installation instructions there are outdated. Maybe that will change. Feel free to contribute if you want.

Max had already given a nice tutorial for his package at the October 2019 B2GM.
In his talk he gave a very nice overview on the theory of template fitting, so I recommend you to look at his [slides](https://indico.belle2.org/event/1158/contributions/4726/attachments/2809/4241/b2gm_templatefitter.pdf). If you want to try out Max' `TemplateFitter`, e.g. to compare it to `binfit`, there's already good [tutorials](https://github.com/welschma/TemplateFitter/blob/master/examples/basic_example.ipynb) in his packages `examples` folder. Also, his package has a nice sphinx [online-documentation on readthedocs](https://templatefitter.readthedocs.io/en/latest/index.html).

## Tutorial author

[Michael Eliachevitch](mailto:meliache@uni-bonn.de "email")


## Installation

### Variant 1: pip install via single command
The fastest is to just install it via `pip` with a single command (if you haven't installed it yet, you can just execute the notebook cell below):

In [ ]:
!python3 -m pip install --user --upgrade "git+ssh://git@stash.desy.de:7999/~sutclw/binfit.git"

### Variant 2: Clone the repository (recommended)

I would recommend thi options, because that makes it actually easier to navigate the source code locally.
Also, you will be able to browse the documentation and examples which come with the package.

```bash
git clone ssh://git@stash.desy.de:7999/~sutclw/binfit.git
cd binfit
python3 -m pip3 install --user --editable --upgrade . 
```

The last install command will install the package and its requirements. The `--editable` changes the installation so that it is performed by symlinking the package files instead of copying them. As a result, if you change something in the source code, the changes will immediately effect the installed version.

## Usage
As soon as the package is installed, you should be able to successfully import it:

In [ ]:
import binfit
import pandas as pd
import numpy as pd